In [9]:
import numpy as np
import threading
import time
import tensorflow as tf

In [ ]:
# QueueRunner 用于启动多个线程操作同一个队列

In [15]:
# 1. 先进先出的队列，容量100，类型为实数
queue = tf.FIFOQueue(100, 'float')
# 2. 定义队列的入队操作
enqueue_op = queue.enqueue([tf.random_normal([1])])
# 表示需要启动5个线程，每个线程中运行的是enqueue_op操作
# tf.train.QueueRunner 第一个参数给出了被操作的队列
# [enqueue_op] * 5 表示需要启动5个线程，每个线程中运行的是enqueue_op操作
# 3.
qr = tf.train.QueueRunner(queue, [enqueue_op] * 5)

In [16]:
# 4. 将定义过的QueueRunner加入tensorflow计算图上指定的集合
# add_queue_runner函数没有指定集合，则默认加入tf.GraphKeys.Queue_RUNNERS集合
tf.train.add_queue_runner(qr)
# 2. 定义队列的出队操作
out_tensor = queue.dequeue()

In [21]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    # 5.启动tf.GraphKeys.Queue_RUNNERS中的QueueRunner
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(3):
        print(sess.run(out_tensor)[0])
    coord.request_stop()
    coord.join(threads)

0.032799784
-0.58927846
-0.10025525
